# VN Training Data Cleaning - Remove Multi-Character Dialogue

**Purpose:** Clean VN training data to remove multi-character conversations and formatting artifacts

**Problem Identified:**
- Training data contains multi-character dialogue scenes (not single-character conversations)
- Assistant responses mention other characters (Monika, Sayori, Natsuki, Yuri)
- VN formatting artifacts present: `{i}...{/i}`, `{b}...{/b}`, etc.
- Model learns to continue multi-character scenes instead of single responses

**What this notebook does:**
1. 📚 Load raw VN JSONL data (all 4 characters)
2. 🔍 Identify multi-character conversations
3. 🧹 Filter out examples where:
   - User mentions other character names
   - Assistant mentions other character names
4. ✨ Clean formatting artifacts:
   - Remove `{i}`, `{/i}`, `{b}`, `{/b}`, etc.
   - Keep `<USER>` placeholder (intentional for inference)
   - Clean excessive ellipsis
5. 📊 Generate before/after statistics
6. 💾 Save cleaned data to `data/processed/VN/cleaned/`

---

## 1. Setup and Imports

In [1]:
import json
import re
from pathlib import Path
import pandas as pd
from collections import defaultdict

print("✓ Imports successful")

✓ Imports successful


## 2. Configuration

In [2]:
# Paths (NO EMOTION version)
VN_DATA_DIR = Path("../../data/processed/VN_no_emotion")
OUTPUT_DIR = Path("../../data/processed/VN_no_emotion/processed")

# Create output directory
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

# Characters
VN_CHARACTERS = ['Monika', 'Sayori', 'Natsuki', 'Yuri']

print(f"Input directory: {VN_DATA_DIR}")
print(f"Output directory: {OUTPUT_DIR}")
print(f"Characters to process: {', '.join(VN_CHARACTERS)}")

Input directory: ../../data/processed/VN_no_emotion
Output directory: ../../data/processed/VN_no_emotion/processed
Characters to process: Monika, Sayori, Natsuki, Yuri


## 3. Define Cleaning Functions

In [3]:
def clean_text_formatting(text):
    """
    Remove VN formatting artifacts from text.
    
    Removes:
    - {i}, {/i}, {b}, {/b}, etc. (VN markup)
    - Excessive ellipsis (more than 3 dots)
    - Extra whitespace
    
    Keeps:
    - <USER> placeholder (intentional for inference)
    """
    # Remove VN markup tags
    text = re.sub(r'\{/?i\}', '', text)  # {i} and {/i}
    text = re.sub(r'\{/?b\}', '', text)  # {b} and {/b}
    text = re.sub(r'\{/?u\}', '', text)  # {u} and {/u}
    text = re.sub(r'\{/?s\}', '', text)  # {s} and {/s}
    text = re.sub(r'\{/?color[^}]*\}', '', text)  # {color=...} and {/color}
    text = re.sub(r'\{/?size[^}]*\}', '', text)  # {size=...} and {/size}
    
    # Clean excessive ellipsis (keep max 3 dots)
    text = re.sub(r'\.{4,}', '...', text)
    
    # Clean up extra whitespace
    text = ' '.join(text.split())
    
    return text.strip()


def contains_other_characters(text, current_character, characters_list):
    """
    Check if text mentions other character names.
    
    Args:
        text: Text to check
        current_character: The character this conversation is about
        characters_list: List of all character names
    
    Returns:
        True if other character names are found, False otherwise
    """
    other_characters = [c for c in characters_list if c != current_character]
    
    for other_char in other_characters:
        # Case-insensitive search for character names
        if re.search(rf'\b{re.escape(other_char)}\b', text, re.IGNORECASE):
            return True
    
    return False


def should_keep_example(messages, current_character, characters_list):
    """
    Determine if a conversation example should be kept.
    
    Filter out if:
    - User turns mention other character names
    - Assistant turns mention other character names
    - No assistant responses present
    
    Args:
        messages: List of message dicts with 'role' and 'content'
        current_character: The character this conversation is about
        characters_list: List of all character names
    
    Returns:
        (keep: bool, reason: str)
    """
    has_assistant = False
    
    for msg in messages:
        role = msg.get('role', '')
        content = msg.get('content', '')
        
        if role == 'assistant':
            has_assistant = True
            # Check if assistant mentions other characters
            if contains_other_characters(content, current_character, characters_list):
                return False, "assistant mentions other characters"
        
        elif role == 'user':
            # Check if user mentions other characters
            if contains_other_characters(content, current_character, characters_list):
                return False, "user mentions other characters"
    
    if not has_assistant:
        return False, "no assistant responses"
    
    return True, "OK"


def clean_example(example, current_character):
    """
    Clean formatting artifacts from all messages in an example.
    
    Args:
        example: Dict with 'messages' key
        current_character: Character name for this conversation
    
    Returns:
        Cleaned example dict
    """
    cleaned_messages = []
    
    for msg in example['messages']:
        cleaned_msg = msg.copy()
        cleaned_msg['content'] = clean_text_formatting(msg['content'])
        cleaned_messages.append(cleaned_msg)
    
    return {'messages': cleaned_messages}


print("✓ Cleaning functions defined")

✓ Cleaning functions defined


## 4. Test Cleaning Functions

In [4]:
# Test text cleaning
test_texts = [
    "Welcome to the {i}Literature Club{/i}! It's a pleasure meeting you.",
    "I-I don't mind... I just need some time........ to think before making any decisions--",
    "Natsuki! What did you just say...?!",
    "The level of {b}creativity{/b} and craftsmanship behind them is amazing.",
]

print("="*80)
print("Text Cleaning Test")
print("="*80)

for text in test_texts:
    cleaned = clean_text_formatting(text)
    print(f"\nOriginal: {text}")
    print(f"Cleaned:  {cleaned}")

# Test character detection
print("\n" + "="*80)
print("Character Mention Detection Test")
print("="*80)

test_cases = [
    ("Welcome to the Literature Club!", "Yuri", False),
    ("Natsuki will be performing tonight!!", "Sayori", True),
    ("I think monika would like this", "Yuri", True),
    ("What do you think about my poem?", "Monika", False),
]

for text, char, expected in test_cases:
    result = contains_other_characters(text, char, VN_CHARACTERS)
    status = "✓" if result == expected else "✗"
    print(f"\n{status} Text: '{text}'")
    print(f"  Character: {char}")
    print(f"  Expected: {expected}, Got: {result}")

Text Cleaning Test

Original: Welcome to the {i}Literature Club{/i}! It's a pleasure meeting you.
Cleaned:  Welcome to the Literature Club! It's a pleasure meeting you.

Original: I-I don't mind... I just need some time........ to think before making any decisions--
Cleaned:  I-I don't mind... I just need some time... to think before making any decisions--

Original: Natsuki! What did you just say...?!
Cleaned:  Natsuki! What did you just say...?!

Original: The level of {b}creativity{/b} and craftsmanship behind them is amazing.
Cleaned:  The level of creativity and craftsmanship behind them is amazing.

Character Mention Detection Test

✓ Text: 'Welcome to the Literature Club!'
  Character: Yuri
  Expected: False, Got: False

✓ Text: 'Natsuki will be performing tonight!!'
  Character: Sayori
  Expected: True, Got: True

✓ Text: 'I think monika would like this'
  Character: Yuri
  Expected: True, Got: True

✓ Text: 'What do you think about my poem?'
  Character: Monika
  Expected: Fal

## 5. Load and Process Data

In [5]:
# Statistics tracking
stats = {
    'total_before': 0,
    'total_after': 0,
    'filtered_reasons': defaultdict(int),
    'by_character': {}
}

print("Processing VN training data...\n")
print("="*80)

for character in VN_CHARACTERS:
    print(f"\nProcessing {character}...")
    
    # Load data
    input_file = VN_DATA_DIR / f"vn_training_data_{character}.jsonl"
    
    with open(input_file, 'r', encoding='utf-8') as f:
        data = [json.loads(line) for line in f]
    
    original_count = len(data)
    stats['total_before'] += original_count
    
    # Filter and clean
    kept_examples = []
    char_filtered_reasons = defaultdict(int)
    
    for example in data:
        keep, reason = should_keep_example(example['messages'], character, VN_CHARACTERS)
        
        if keep:
            # Clean the example
            cleaned = clean_example(example, character)
            kept_examples.append(cleaned)
        else:
            char_filtered_reasons[reason] += 1
            stats['filtered_reasons'][reason] += 1
    
    kept_count = len(kept_examples)
    stats['total_after'] += kept_count
    filtered_count = original_count - kept_count
    
    # Save cleaned data
    output_file = OUTPUT_DIR / f"vn_training_data_{character}_cleaned.jsonl"
    
    with open(output_file, 'w', encoding='utf-8') as f:
        for example in kept_examples:
            f.write(json.dumps(example, ensure_ascii=False) + '\n')
    
    # Store character stats
    stats['by_character'][character] = {
        'original': original_count,
        'kept': kept_count,
        'filtered': filtered_count,
        'filtered_reasons': dict(char_filtered_reasons)
    }
    
    # Print character summary
    print(f"  Original: {original_count} examples")
    print(f"  Kept:     {kept_count} examples ({kept_count/original_count*100:.1f}%)")
    print(f"  Filtered: {filtered_count} examples ({filtered_count/original_count*100:.1f}%)")
    
    if char_filtered_reasons:
        print(f"  Reasons:")
        for reason, count in char_filtered_reasons.items():
            print(f"    - {reason}: {count}")
    
    print(f"  ✓ Saved to: {output_file}")

print("\n" + "="*80)
print("✓ Processing complete!")

Processing VN training data...


Processing Monika...
  Original: 110 examples
  Kept:     29 examples (26.4%)
  Filtered: 81 examples (73.6%)
  Reasons:
    - user mentions other characters: 48
    - assistant mentions other characters: 33
  ✓ Saved to: ../../data/processed/VN_no_emotion/processed/vn_training_data_Monika_cleaned.jsonl

Processing Sayori...
  Original: 90 examples
  Kept:     49 examples (54.4%)
  Filtered: 41 examples (45.6%)
  Reasons:
    - assistant mentions other characters: 18
    - user mentions other characters: 23
  ✓ Saved to: ../../data/processed/VN_no_emotion/processed/vn_training_data_Sayori_cleaned.jsonl

Processing Natsuki...
  Original: 111 examples
  Kept:     52 examples (46.8%)
  Filtered: 59 examples (53.2%)
  Reasons:
    - user mentions other characters: 42
    - assistant mentions other characters: 17
  ✓ Saved to: ../../data/processed/VN_no_emotion/processed/vn_training_data_Natsuki_cleaned.jsonl

Processing Yuri...
  Original: 128 examples
  Ke

## 6. Overall Statistics

In [6]:
print("="*80)
print("OVERALL STATISTICS")
print("="*80)

print(f"\nTotal examples:")
print(f"  Before: {stats['total_before']}")
print(f"  After:  {stats['total_after']}")
print(f"  Filtered: {stats['total_before'] - stats['total_after']} ({(stats['total_before'] - stats['total_after'])/stats['total_before']*100:.1f}%)")

print(f"\nFiltering reasons (overall):")
for reason, count in sorted(stats['filtered_reasons'].items(), key=lambda x: -x[1]):
    pct = count / stats['total_before'] * 100
    print(f"  {reason}: {count} ({pct:.1f}%)")

print(f"\nPer-character breakdown:")
for character in VN_CHARACTERS:
    char_stats = stats['by_character'][character]
    retention_rate = char_stats['kept'] / char_stats['original'] * 100
    print(f"  {character:12s}: {char_stats['kept']:3d}/{char_stats['original']:3d} kept ({retention_rate:.1f}% retention)")

OVERALL STATISTICS

Total examples:
  Before: 439
  After:  200
  Filtered: 239 (54.4%)

Filtering reasons (overall):
  user mentions other characters: 152 (34.6%)
  assistant mentions other characters: 87 (19.8%)

Per-character breakdown:
  Monika      :  29/110 kept (26.4% retention)
  Sayori      :  49/ 90 kept (54.4% retention)
  Natsuki     :  52/111 kept (46.8% retention)
  Yuri        :  70/128 kept (54.7% retention)


## 7. Sample Comparisons

In [7]:
print("="*80)
print("SAMPLE CLEANED CONVERSATIONS")
print("="*80)

# Show 2 examples from each character
for character in VN_CHARACTERS:
    print(f"\n{'─'*80}")
    print(f"{character} - Sample Cleaned Conversations")
    print(f"{'─'*80}")
    
    # Load cleaned data
    cleaned_file = OUTPUT_DIR / f"vn_training_data_{character}_cleaned.jsonl"
    
    with open(cleaned_file, 'r', encoding='utf-8') as f:
        cleaned_data = [json.loads(line) for line in f]
    
    # Show first 2 examples
    for i, example in enumerate(cleaned_data[:2], 1):
        print(f"\nExample {i}:")
        for msg in example['messages']:
            role = msg['role']
            content = msg['content'][:100] + "..." if len(msg['content']) > 100 else msg['content']
            print(f"  {role:10s}: {content}")

print("\n" + "="*80)

SAMPLE CLEANED CONVERSATIONS

────────────────────────────────────────────────────────────────────────────────
Monika - Sample Cleaned Conversations
────────────────────────────────────────────────────────────────────────────────

Example 1:
  system    : You are Monika, the Literature Club president. Confident, intelligent, and caring. You're thoughtful...
  user      : Don't make promises you can't keep! Fine... I'll stop by for a cupcake, okay? I told you, don't call...
  assistant : Ah, <USER>! What a nice surprise! Welcome to the club!
  user      : ... S-Sorry... Ah... Well, it's nice to meet both of you.
  assistant : That's right. It's great to see you again, <USER>.
  user      : Y-<USER> too, Monika.

Example 2:
  system    : You are Monika, the Literature Club president. Confident, intelligent, and caring. You're thoughtful...
  user      : Okay, I've decided, then. I'll join the Literature Club. H-Hey--
  assistant : Welcome to the Literature Club!
  user      : Ah...thanks

## 8. Save Statistics Report

In [8]:
# Save statistics to JSON
stats_file = OUTPUT_DIR / "cleaning_statistics.json"

# Convert defaultdict to regular dict for JSON serialization
stats_to_save = {
    'total_before': stats['total_before'],
    'total_after': stats['total_after'],
    'filtered_reasons': dict(stats['filtered_reasons']),
    'by_character': stats['by_character']
}

with open(stats_file, 'w', encoding='utf-8') as f:
    json.dump(stats_to_save, f, indent=2, ensure_ascii=False)

print(f"✓ Statistics saved to: {stats_file}")

# Create summary text file
summary_file = OUTPUT_DIR / "CLEANING_SUMMARY.txt"

with open(summary_file, 'w', encoding='utf-8') as f:
    f.write("VN TRAINING DATA CLEANING SUMMARY\n")
    f.write("="*80 + "\n\n")
    
    f.write("OVERALL STATISTICS\n")
    f.write("-"*80 + "\n")
    f.write(f"Total examples before: {stats['total_before']}\n")
    f.write(f"Total examples after:  {stats['total_after']}\n")
    f.write(f"Filtered:              {stats['total_before'] - stats['total_after']} ({(stats['total_before'] - stats['total_after'])/stats['total_before']*100:.1f}%)\n\n")
    
    f.write("FILTERING REASONS\n")
    f.write("-"*80 + "\n")
    for reason, count in sorted(stats['filtered_reasons'].items(), key=lambda x: -x[1]):
        pct = count / stats['total_before'] * 100
        f.write(f"  {reason}: {count} ({pct:.1f}%)\n")
    
    f.write("\nPER-CHARACTER BREAKDOWN\n")
    f.write("-"*80 + "\n")
    for character in VN_CHARACTERS:
        char_stats = stats['by_character'][character]
        retention_rate = char_stats['kept'] / char_stats['original'] * 100
        f.write(f"\n{character}:\n")
        f.write(f"  Original: {char_stats['original']}\n")
        f.write(f"  Kept:     {char_stats['kept']} ({retention_rate:.1f}% retention)\n")
        f.write(f"  Filtered: {char_stats['filtered']}\n")
        
        if char_stats['filtered_reasons']:
            f.write(f"  Reasons:\n")
            for reason, count in char_stats['filtered_reasons'].items():
                f.write(f"    - {reason}: {count}\n")
    
    f.write("\n" + "="*80 + "\n")
    f.write("\nNEXT STEPS:\n")
    f.write("1. Review sample cleaned conversations above\n")
    f.write("2. Update training notebook to use cleaned data:\n")
    f.write("   VN_DATA_DIR = '../../data/processed/VN/cleaned'\n")
    f.write("3. Retrain model with cleaned data\n")
    f.write("4. Verify character blending is reduced\n")

print(f"✓ Summary saved to: {summary_file}")

print("\n" + "="*80)
print("✅ DATA CLEANING COMPLETE!")
print("="*80)
print(f"\nCleaned data location: {OUTPUT_DIR}")
print(f"Total cleaned examples: {stats['total_after']}")
print(f"\nNext steps:")
print("1. Review the cleaned data samples above")
print("2. Update training notebook (03_complete_training__VN.ipynb):")
print("   Change: VN_DATA_DIR = '../../data/processed/VN/cleaned'")
print("3. Retrain model with cleaned data")
print("4. Test generation with updated parameters (max_new_tokens=50)")

✓ Statistics saved to: ../../data/processed/VN_no_emotion/processed/cleaning_statistics.json
✓ Summary saved to: ../../data/processed/VN_no_emotion/processed/CLEANING_SUMMARY.txt

✅ DATA CLEANING COMPLETE!

Cleaned data location: ../../data/processed/VN_no_emotion/processed
Total cleaned examples: 200

Next steps:
1. Review the cleaned data samples above
2. Update training notebook (03_complete_training__VN.ipynb):
   Change: VN_DATA_DIR = '../../data/processed/VN/cleaned'
3. Retrain model with cleaned data
4. Test generation with updated parameters (max_new_tokens=50)
